In [2]:
import pandas as pd
import json
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder
import sys
import numpy as np

In [3]:
data = pd.read_csv('rMinerRefactorings.csv')
data.shape

(904844, 7)

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

np.set_printoptions(threshold=sys.maxsize)

In [5]:
data.isna().sum()

_id                   0
commit_id             0
description           0
leftSideLocations     0
rightSideLocations    0
test                  0
type                  0
dtype: int64

In [6]:
data.head(2)

,_id,commit_id,description,leftSideLocations,rightSideLocations,test,type
0,61d25008dc748d43d2fdf094,5a85632d77d2b433df71b6b2,Rename Method public testThinRectangle() : voi...,"[{""filePath"":""src/test/java/org/apache/commons...","[{""filePath"":""src/test/java/org/apache/commons...",True,Rename Method
1,61d2500adc748d43d2fdf095,5a85632d77d2b433df71b6b2,Extract Attribute private TEST_TOLERANCE : dou...,"[{""filePath"":""src/test/java/org/apache/commons...","[{""filePath"":""src/test/java/org/apache/commons...",True,Extract Attribute


# Parse the "rightSideLocations" and "leftSideLocations" fileds

In [7]:
data['rightSideLocations'] = data['rightSideLocations'].apply(lambda x: json.loads(x))
data['leftSideLocations'] = data['leftSideLocations'].apply(lambda x: json.loads(x))

# Refactory level features

In [8]:
def clac_sum_of_diffs(side_location_info: str):
    location_count = len(side_location_info)
    diff_list = []
    for i in range(location_count):
        start_line = side_location_info[i]['startLine']
        end_line = side_location_info[i]['endLine']
        if type(start_line) != int or type(end_line) != int:
            print('non integer type')
        else:
            diff = (end_line - start_line) + 1
            diff_list.append(diff)
    return sum(diff_list)
    
        

In [9]:
def file_touch(side_location_info: str):
    location_count = len(side_location_info)
    file_names = []
    for i in range(location_count):
        file_name = side_location_info[i]['filePath']

        if len(file_name) == 0:
            print('There is no file names.')
        else: 
            file_names.append(file_name)
    return list(set(file_names))

In [10]:
def get_code_element_types(side_location_info: str):
    location_count = len(side_location_info)
    code_element_types = []
    for i in range(location_count):
        code_element = side_location_info[i]['codeElementType']

        if len(code_element) == 0:
            print('There is no code element in the current location.')
        else: 
            code_element_types.append(code_element)
    return code_element_types

### If refactory type is "Move Source Folder", the number of touched files will be 0

In [11]:
data['countRightSideLocations'] = data['rightSideLocations'].apply(lambda x: len(x))
data['countLeftSideLocations'] = data['leftSideLocations'].apply(lambda x: len(x))

data['sumOfDiffLeftSide'] = data['leftSideLocations'].apply(lambda x: clac_sum_of_diffs(side_location_info = x))
data['sumOfDiffRightSide'] = data['rightSideLocations'].apply(lambda x: clac_sum_of_diffs(side_location_info = x))

data['fileTouchLeftSide'] = data['leftSideLocations'].apply(lambda x: file_touch(side_location_info = x))
data['fileTouchRightSide'] = data['rightSideLocations'].apply(lambda x: file_touch(side_location_info = x))

data['fileTouch'] = data['fileTouchLeftSide'] + data['fileTouchRightSide']
data['fileTouch'] = data['fileTouch'].apply(lambda x: list(set(x)))

data['fileTouchCount'] = data['fileTouch'].apply(lambda x: len(x))

data['codeElementTypeRightSide'] = data['rightSideLocations'].apply(lambda x: get_code_element_types(side_location_info = x))
data['codeElementTypeLeftSide'] = data['leftSideLocations'].apply(lambda x: get_code_element_types(side_location_info = x))

data['codeElementType'] = data['codeElementTypeRightSide'] + data['codeElementTypeLeftSide']

data['refType'] = data['type'] # to keep a copy of "type" column
data = pd.get_dummies(data, columns = ['refType'])

codeElementType_flattend = data['codeElementType'].explode()
print('Filling NaN with 0.')
data = data.join(pd.crosstab(index=codeElementType_flattend.index, columns=codeElementType_flattend)).fillna(0)



Filling NaN with 0.


In [12]:
data.head(10)#['fileTouch'].apply(lambda x: print(x))# if len(x)>1 else print('no'))

,_id,commit_id,description,leftSideLocations,rightSideLocations,test,type,countRightSideLocations,countLeftSideLocations,sumOfDiffLeftSide,sumOfDiffRightSide,fileTouchLeftSide,fileTouchRightSide,fileTouch,fileTouchCount,codeElementTypeRightSide,codeElementTypeLeftSide,codeElementType,refType_Add Attribute Annotation,refType_Add Attribute Modifier,refType_Add Class Annotation,refType_Add Class Modifier,refType_Add Method Annotation,refType_Add Method Modifier,refType_Add Parameter,refType_Add Parameter Annotation,refType_Add Parameter Modifier,refType_Add Thrown Exception Type,refType_Add Variable Annotation,refType_Add Variable Modifier,refType_Change Attribute Access Modifier,refType_Change Attribute Type,refType_Change Class Access Modifier,refType_Change Method Access Modifier,refType_Change Parameter Type,refType_Change Return Type,refType_Change Thrown Exception Type,refType_Change Type Declaration Kind,refType_Change Variable Type,refType_Collapse Hierarchy,refType_Encapsulate Attribute,refType_Extract And Move Method,refType_Extract Attribute,refType_Extract Class,refType_Extract Interface,refType_Extract Method,refType_Extract Subclass,refType_Extract Superclass,refType_Extract Variable,refType_Inline Method,refType_Inline Variable,refType_Localize Parameter,refType_Merge Attribute,refType_Merge Package,refType_Merge Parameter,refType_Merge Variable,refType_Modify Attribute Annotation,refType_Modify Class Annotation,refType_Modify Method Annotation,refType_Modify Parameter Annotation,refType_Modify Variable Annotation,refType_Move And Inline Method,refType_Move And Rename Attribute,refType_Move And Rename Class,refType_Move And Rename Method,refType_Move Attribute,refType_Move Class,refType_Move Method,refType_Move Package,refType_Move Source Folder,refType_Parameterize Attribute,refType_Parameterize Variable,refType_Pull Up Attribute,refType_Pull Up Method,refType_Push Down Attribute,refType_Push Down Method,refType_Remove Attribute Annotation,refType_Remove Attribute Modifier,refType_Remove Class Annotation,refType_Remove Class Modifier,refType_Remove Method Annotation,refType_Remove Method Modifier,refType_Remove Parameter,refType_Remove Parameter Annotation,refType_Remove Parameter Modifier,refType_Remove Thrown Exception Type,refType_Remove Variable Annotation,refType_Remove Variable Modifier,refType_Rename Attribute,refType_Rename Class,refType_Rename Method,refType_Rename Package,refType_Rename Parameter,refType_Rename Variable,refType_Reorder Parameter,refType_Replace Anonymous With Lambda,refType_Replace Attribute,refType_Replace Attribute With Variable,refType_Replace Loop With Pipeline,refType_Replace Variable With Attribute,refType_Split Attribute,refType_Split Package,refType_Split Parameter,refType_Split Variable,ANNOTATION,ANONYMOUS_CLASS_DECLARATION,ASSERT_STATEMENT,BLOCK,BREAK_STATEMENT,CATCH_CLAUSE,CATCH_CLAUSE_EXCEPTION_NAME,CONSTRUCTOR_INVOCATION,CONTINUE_STATEMENT,DO_STATEMENT,DO_STATEMENT_CONDITION,EMPTY_STATEMENT,ENHANCED_FOR_STATEMENT,ENHANCED_FOR_STATEMENT_EXPRESSION,ENHANCED_FOR_STATEMENT_PARAMETER_NAME,ENUM_CONSTANT_DECLARATION,EXPRESSION_STATEMENT,FIELD_DECLARATION,FINALLY_BLOCK,FOR_STATEMENT,FOR_STATEMENT_CONDITION,FOR_STATEMENT_INITIALIZER,FOR_STATEMENT_UPDATER,IF_STATEMENT,IF_STATEMENT_CONDITION,LABELED_STATEMENT,LAMBDA_EXPRESSION,LAMBDA_EXPRESSION_BODY,METHOD_DECLARATION,METHOD_INVOCATION,RETURN_STATEMENT,SINGLE_VARIABLE_DECLARATION,SUPER_CONSTRUCTOR_INVOCATION,SUPER_METHOD_INVOCATION,SWITCH_CASE,SWITCH_STATEMENT,SWITCH_STATEMENT_CONDITION,SYNCHRONIZED_STATEMENT,SYNCHRONIZED_STATEMENT_EXPRESSION,THROW_STATEMENT,TRY_STATEMENT,TRY_STATEMENT_RESOURCE,TYPE,TYPE_DECLARATION,VARIABLE_DECLARATION_EXPRESSION,VARIABLE_DECLARATION_STATEMENT,WHILE_STATEMENT,WHILE_STATEMENT_CONDITION
0,61d25008dc748d43d2fdf094,5a85632d77d2b433df71b6b2,Rename Method public testThinRectangle() : voi...,[{'filePath': 'src/test/java/org/apache/common...,[{'filePath': 'src/test/java/org/apache/common...,True,Rename Metho

In [13]:
data.head(10)[['commit_id', 'fileTouch']]


,commit_id,fileTouch
0,5a85632d77d2b433df71b6b2,[src/test/java/org/apache/commons/math4/geomet...
1,5a85632d77d2b433df71b6b2,[src/test/java/org/apache/commons/math4/geomet...
2,5a85632d77d2b433df71b6b2,[src/test/java/org/apache/commons/math4/geomet...
3,5a85632d77d2b433df71b6b2,[src/test/java/org/apache/commons/math4/geomet...
4,5a85632d77d2b433df71b6b2,[src/test/java/org/apache/commons/math4/geomet...
5,5a85632d77d2b433df71b6b2,[src/test/java/org/apache/commons/math4/geomet...
6,5a85632d77d2b433df71b6b2,[src/test/java/org/apache/commons/math4/geomet...
7,5a85632d77d2b433df71b6b2,[src/test/java/org/apache/commons/math4/geomet...
8,5a85632d77d2b433df71b6b2,[src/test/java/org/apache/commons/math4/geomet...
9,5a8562cc77d2b433dc711f29,[src/test/java/org/apache/commons/math4/geomet...


In [27]:
print('Total number of NaN values: ', data.isna().sum().sum())
data.isna().sum()

Total number of NaN values:  0


_id                                         0
commit_id                                   0
description                                 0
leftSideLocations                           0
rightSideLocations                          0
test                                        0
type                                        0
countRightSideLocations                     0
countLeftSideLocations                      0
sumOfDiffLeftSide                           0
sumOfDiffRightSide                          0
fileTouchLeftSide                           0
fileTouchRightSide                          0
fileTouch                                   0
fileTouchCount                              0
codeElementTypeRightSide                    0
codeElementTypeLeftSide                     0
codeElementType                             0
refType_Add Attribute Annotation            0
refType_Add Attribute Modifier              0
refType_Add Class Annotation                0
refType_Add Class Modifier        

In [18]:
# Save the Refactory level features as a csv file
data.to_csv('feature_ref_level.csv', index=False)

# Commit level features

In [19]:
def get_refType_feature_names(ref_type_column: pd.Series):
    ref_type = list(ref_type_column.unique())
    ref_type_names = []
    
    for i in range(len(ref_type)):
        ref_type_names.append('refType_' + (ref_type[i]))
    return ref_type_names

In [20]:
def label_generator(isTest_unique_answer: list):
    if len(isTest_unique_answer) == 2 and True in isTest_unique_answer and False in isTest_unique_answer:
        return 'co-occur'

    elif len(isTest_unique_answer) == 1:
        if True in isTest_unique_answer:
            return 'test_only'
        elif False in isTest_unique_answer:
            return 'prod_only'
        
    else:
        print('There is a problem with the isTest unique list.')
        return None

In [21]:
# add the labels
commit_isTest = pd.DataFrame(data.groupby('commit_id', sort=False)['test'].unique()) # labels
commit_isTest['label'] = commit_isTest['test'].apply(lambda x: label_generator(isTest_unique_answer=x))
# commit_isTest = commit_isTest.drop(columns='isTest')
commit_isTest_false = commit_isTest[commit_isTest['label']!="test_only"]
commit_isTest_false.head(2)
print(commit_isTest.shape)
print(commit_isTest_false.shape)

(53829, 2)
(49676, 2)


In [22]:
49676/53829

0.9228482788088205

# Edit

This cell includes the modification, in order to include the full raw data with sole test code refactorings commits. The variable name was therefore changed from "no_test_data" (in the original file) to "inc_test_data".

In [23]:
# Edit: Removed the query, where the test code refactorings were filtered away.
inc_test_data = data #.query('test == False ')
inc_test_data.head(10)

commit_ref_count = inc_test_data.groupby('commit_id', sort=False)['_id'].count() # number of refactorings with respect to unique ref ID

commit_left_loc_count = inc_test_data.groupby('commit_id', sort=False)['countLeftSideLocations'].sum()
commit_right_loc_count = inc_test_data.groupby('commit_id', sort=False)['countRightSideLocations'].sum()

commit_left_loc_diff = inc_test_data.groupby('commit_id', sort=False)['sumOfDiffLeftSide'].sum()
commit_right_loc_diff = inc_test_data.groupby('commit_id', sort=False)['sumOfDiffRightSide'].sum()

# aggregating the touch files in left and right side locations in commit level (agg left and right for all refactorings
# of one commit), remove the duplicate ones, count the number of the unique ones.
commit_fileTouch_count = inc_test_data.groupby('commit_id', sort=False).agg({'fileTouch': lambda x: len(set(list(np.hstack(x))))})

commint_fileTouchCount_avg = inc_test_data.groupby('commit_id', sort=False)['fileTouchCount'].mean()

refType_name = get_refType_feature_names(ref_type_column=inc_test_data['type'])
commit_refType_count = inc_test_data.groupby('commit_id', sort=False)[refType_name].sum()

codeElement_names = inc_test_data['codeElementType'].explode().dropna().unique()
commit_codeElement_count = inc_test_data.groupby('commit_id', sort=False)[codeElement_names].sum()

commit_uniqueRefType_count = inc_test_data.groupby('commit_id', sort=False)['type'].nunique()

commit_uniqueCodeElementType_count = inc_test_data.groupby('commit_id', sort=False).agg({'codeElementType': lambda x: len(set(list(np.hstack(x))))})




In [24]:
commit_level_features_set1 = pd.concat([commit_ref_count,
                                   commit_left_loc_count,
                                   commit_right_loc_count,
                                   commit_left_loc_diff,
                                   commit_right_loc_diff,
                                   commit_fileTouch_count,
                                   commint_fileTouchCount_avg,
                                   commit_uniqueCodeElementType_count, 
                                   commit_uniqueRefType_count,
                                  ], axis=1)
commit_level_features_set1.columns = ['refactoryCount', 'leftLocationCount', 'rightLocationCount',
                                'leftLocationDiff', 'rightLocationDiff', 'fileTouchCount',
                                'fileTouchAverage', 'codeElementCount', 'refactoryTypeCount']

commit_level_features = pd.concat([commit_level_features_set1, commit_refType_count, commit_codeElement_count, commit_isTest_false], axis=1)

commit_level_features.reset_index(level=0, inplace=True)

# add label column
# commit_level_features['label'] = commit_level_features['isTest'].apply(lambda x: label_generator(isTest_unique_answer=x))
# commit_level_features = commit_level_features.drop(columns='isTest')

commit_level_features.head(100)


,commit_id,refactoryCount,leftLocationCount,rightLocationCount,leftLocationDiff,rightLocationDiff,fileTouchCount,fileTouchAverage,codeElementCount,refactoryTypeCount,refType_Rename Method,refType_Extract Attribute,refType_Extract Variable,refType_Remove Method Annotation,refType_Change Parameter Type,refType_Rename Parameter,refType_Extract And Move Method,refType_Rename Variable,refType_Remove Thrown Exception Type,refType_Extract Method,refType_Add Parameter,refType_Change Method Access Modifier,refType_Change Return Type,refType_Change Variable Type,refType_Change Attribute Type,refType_Add Class Modifier,refType_Extract Superclass,refType_Add Method Annotation,refType_Rename Class,refType_Inline Variable,refType_Add Variable Modifier,refType_Remove Variable Modifier,refType_Add Parameter Modifier,refType_Remove Parameter,refType_Add Attribute Modifier,refType_Move Class,refType_Remove Attribute Modifier,refType_Rename Attribute,refType_Remove Parameter Modifier,refType_Add Method Modifier,refType_Add Thrown Exception Type,refType_Move Method,refType_Remove Class Modifier,refType_Remove Attribute Annotation,refType_Move And Inline Method,refType_Inline Method,refType_Change Attribute Access Modifier,refType_Add Attribute Annotation,refType_Remove Method Modifier,refType_Replace Variable With Attribute,refType_Encapsulate Attribute,refType_Parameterize Variable,refType_Modify Method Annotation,refType_Pull Up Method,refType_Extract Interface,refType_Change Class Access Modifier,refType_Replace Attribute With Variable,refType_Move Attribute,refType_Remove Parameter Annotation,refType_Modify Variable Annotation,refType_Add Class Annotation,refType_Localize Parameter,refType_Extract Class,refType_Pull Up Attribute,refType_Move And Rename Class,refType_Move And Rename Method,refType_Remove Variable Annotation,refType_Move Source Folder,refType_Merge Variable,refType_Change Thrown Exception Type,refType_Add Variable Annotation,refType_Push Down Method,refType_Merge Parameter,refType_Split Attribute,refType_Remove Class Annotation,refType_Parameterize Attribute,refType_Reorder Parameter,refType_Rename Package,refType_Change Type Declaration Kind,refType_Collapse Hierarchy,refType_Push Down Attribute,refType_Split Parameter,refType_Extract Subclass,refType_Split Variable,refType_Move Package,refType_Merge Attribute,refType_Split Package,refType_Modify Class Annotation,refType_Modify Attribute Annotation,refType_Replace Anonymous With Lambda,refType_Modify Parameter Annotation,refType_Replace Loop With Pipeline,refType_Merge Package,refType_Add Parameter Annotation,refType_Move And Rename Attribute,refType_Replace Attribute,METHOD_DECLARATION,FIELD_DECLARATION,VARIABLE_DECLARATION_STATEMENT,EXPRESSION_STATEMENT,ANNOTATION,SINGLE_VARIABLE_DECLARATION,METHOD_INVOCATION,TYPE,TYPE_DECLARATION,RETURN_STATEMENT,BLOCK,FOR_STATEMENT,IF_STATEMENT,CONTINUE_STATEMENT,WHILE_STATEMENT,THROW_STATEMENT,TRY_STATEMENT,CATCH_CLAUSE,BREAK_STATEMENT,VARIABLE_DECLARATION_EXPRESSION,FOR_STATEMENT_INITIALIZER,ENHANCED_FOR_STATEMENT,DO_STATEMENT,IF_STATEMENT_CONDITION,SYNCHRONIZED_STATEMENT,CONSTRUCTOR_INVOCATION,SUPER_CONSTRUCTOR_INVOCATION,SWITCH_CASE,SWITCH_STATEMENT,ENHANCED_FOR_STATEMENT_EXPRESSION,SUPER_METHOD_INVOCATION,ASSERT_STATEMENT,ENUM_CONSTANT_DECLARATION,CATCH_CLAUSE_EXCEPTION_NAME,FINALLY_BLOCK,EMPTY_STATEMENT,FOR_STATEMENT_UPDATER,LAMBDA_EXPRESSION,ANONYMOUS_CLASS_DECLARATION,SWITCH_STATEMENT_CONDITION,ENHANCED_FOR_STATEMENT_PARAMETER_NAME,LABELED_STATEMENT,FOR_STATEMENT_CONDITION,TRY_STATEMENT_RESOURCE,WHILE_STATEMENT_CONDITION,LAMBDA_EXPRESSION_BODY,SYNCHRONIZED_STATEMENT_EXPRESSION,DO_STATEMENT_CONDITION,test,label
0,5a85632d77d2b433df71b6b2,9,19,29,173,191,2,1.333333,7,7,1,1,1,1,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.0,1.0,6.0,14.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [26]:
commit_level_features.shape
commit_level_features.head(10)

,commit_id,refactoryCount,leftLocationCount,rightLocationCount,leftLocationDiff,rightLocationDiff,fileTouchCount,fileTouchAverage,codeElementCount,refactoryTypeCount,refType_Rename Method,refType_Extract Attribute,refType_Extract Variable,refType_Remove Method Annotation,refType_Change Parameter Type,refType_Rename Parameter,refType_Extract And Move Method,refType_Rename Variable,refType_Remove Thrown Exception Type,refType_Extract Method,refType_Add Parameter,refType_Change Method Access Modifier,refType_Change Return Type,refType_Change Variable Type,refType_Change Attribute Type,refType_Add Class Modifier,refType_Extract Superclass,refType_Add Method Annotation,refType_Rename Class,refType_Inline Variable,refType_Add Variable Modifier,refType_Remove Variable Modifier,refType_Add Parameter Modifier,refType_Remove Parameter,refType_Add Attribute Modifier,refType_Move Class,refType_Remove Attribute Modifier,refType_Rename Attribute,refType_Remove Parameter Modifier,refType_Add Method Modifier,refType_Add Thrown Exception Type,refType_Move Method,refType_Remove Class Modifier,refType_Remove Attribute Annotation,refType_Move And Inline Method,refType_Inline Method,refType_Change Attribute Access Modifier,refType_Add Attribute Annotation,refType_Remove Method Modifier,refType_Replace Variable With Attribute,refType_Encapsulate Attribute,refType_Parameterize Variable,refType_Modify Method Annotation,refType_Pull Up Method,refType_Extract Interface,refType_Change Class Access Modifier,refType_Replace Attribute With Variable,refType_Move Attribute,refType_Remove Parameter Annotation,refType_Modify Variable Annotation,refType_Add Class Annotation,refType_Localize Parameter,refType_Extract Class,refType_Pull Up Attribute,refType_Move And Rename Class,refType_Move And Rename Method,refType_Remove Variable Annotation,refType_Move Source Folder,refType_Merge Variable,refType_Change Thrown Exception Type,refType_Add Variable Annotation,refType_Push Down Method,refType_Merge Parameter,refType_Split Attribute,refType_Remove Class Annotation,refType_Parameterize Attribute,refType_Reorder Parameter,refType_Rename Package,refType_Change Type Declaration Kind,refType_Collapse Hierarchy,refType_Push Down Attribute,refType_Split Parameter,refType_Extract Subclass,refType_Split Variable,refType_Move Package,refType_Merge Attribute,refType_Split Package,refType_Modify Class Annotation,refType_Modify Attribute Annotation,refType_Replace Anonymous With Lambda,refType_Modify Parameter Annotation,refType_Replace Loop With Pipeline,refType_Merge Package,refType_Add Parameter Annotation,refType_Move And Rename Attribute,refType_Replace Attribute,METHOD_DECLARATION,FIELD_DECLARATION,VARIABLE_DECLARATION_STATEMENT,EXPRESSION_STATEMENT,ANNOTATION,SINGLE_VARIABLE_DECLARATION,METHOD_INVOCATION,TYPE,TYPE_DECLARATION,RETURN_STATEMENT,BLOCK,FOR_STATEMENT,IF_STATEMENT,CONTINUE_STATEMENT,WHILE_STATEMENT,THROW_STATEMENT,TRY_STATEMENT,CATCH_CLAUSE,BREAK_STATEMENT,VARIABLE_DECLARATION_EXPRESSION,FOR_STATEMENT_INITIALIZER,ENHANCED_FOR_STATEMENT,DO_STATEMENT,IF_STATEMENT_CONDITION,SYNCHRONIZED_STATEMENT,CONSTRUCTOR_INVOCATION,SUPER_CONSTRUCTOR_INVOCATION,SWITCH_CASE,SWITCH_STATEMENT,ENHANCED_FOR_STATEMENT_EXPRESSION,SUPER_METHOD_INVOCATION,ASSERT_STATEMENT,ENUM_CONSTANT_DECLARATION,CATCH_CLAUSE_EXCEPTION_NAME,FINALLY_BLOCK,EMPTY_STATEMENT,FOR_STATEMENT_UPDATER,LAMBDA_EXPRESSION,ANONYMOUS_CLASS_DECLARATION,SWITCH_STATEMENT_CONDITION,ENHANCED_FOR_STATEMENT_PARAMETER_NAME,LABELED_STATEMENT,FOR_STATEMENT_CONDITION,TRY_STATEMENT_RESOURCE,WHILE_STATEMENT_CONDITION,LAMBDA_EXPRESSION_BODY,SYNCHRONIZED_STATEMENT_EXPRESSION,DO_STATEMENT_CONDITION,test,label
0,5a85632d77d2b433df71b6b2,9,19,29,173,191,2,1.333333,7,7,1,1,1,1,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.0,1.0,6.0,14.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [28]:
commit_level_features.to_csv('commit_level_features_w_test.csv', index=False)